## Plot MatSuMoTo results Vs Master curves (macroscopic creep tests)

### Imports and working directory

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from mat73 import loadmat
from matplotlib.lines import Line2D
from matplotlib.ticker import MaxNLocator
"""plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'stix'"""

cwd = os.getcwd()
print("Working in", os.getcwd())

### Load datasets and show results

In [ ]:
# Set paths
macropath = os.path.join(cwd, 'macro_master_gamma.csv')
outpath = os.path.join(cwd, 'best_gamma.csv')
# Load master curves
df_master = pd.read_csv(macropath).rename(columns={'Unnamed: 0': 'D'})
# Attempt folders
attempt_dirs = sorted(glob.glob(os.path.join(cwd, 'Attempt_*')))

# Unicode subscript map and formatter
sub_map = str.maketrans('0123456789', '₀₁₂₃₄₅₆₇₈₉')
def format_gamma(col_name):
    # expects 'gamma1', 'gamma2', ...
    idx = col_name.replace('gamma', '')
    return f'γ{idx}'.translate(sub_map)

if not attempt_dirs:
    # no attempts, just read the existing summary and rebuild `records`
    print("No Attempt_* folders found. Loading existing best_gamma.csv …")
    df_out = pd.read_csv(outpath)
    gamma_cols = [c for c in df_out.columns if c != 'Material']
    rename_map = {c: format_gamma(c) for c in gamma_cols}
    df_out = df_out.rename(columns=rename_map)
    df_out = df_out.set_index('Material')
    df_out.index.name = None
    display(df_out.round(4))
    # if you need the same `records` list:
    records = df_out.to_dict(orient='records')
else:
    # --- existing MatSuMoTo logic ---
    matfile = 'Results.mat'
    data = loadmat(matfile)['Data']
    xbest = np.array(data['xbest'], dtype=float).flatten()

    # find bestpath
    bestpath = None
    tol = 1e-5
    for attempt in attempt_dirs:
        csv_path = os.path.join(attempt, 'Modules', 'layers_gamma_attempts.csv')
        if not os.path.isfile(csv_path):
            continue
        df = pd.read_csv(csv_path, sep='\t', header=None)
        if np.isclose(df.iloc[:, :4].values, xbest, atol=tol).all(axis=1).any():
            bestpath = attempt
            break

    if bestpath is None:
        raise RuntimeError("No matching Attempt_* folder for xbest!")
    print(f"Best MatSuMoTo attempt folder: {bestpath}")

    # Load growth-ring output
    grpath = os.path.join(bestpath, 'GR', 'GR_gamma.csv')
    df_mat = pd.read_csv(grpath).rename(columns={'Unnamed: 0': 'D'})

    # prepare names
    n_gamma = xbest.size
    gamma_names_greek = [f"γ{i+1}" for i in range(n_gamma)]
    gamma_names_num = [f"gamma{i+1}" for i in range(n_gamma)]
    records = []

    # layers
    df_xbest = pd.DataFrame([xbest], columns=gamma_names_greek)
    print("\nInput values for layers:")
    display(df_xbest.style.format("{:.3f}").hide(axis="index"))
    layers_rec = {"Material": "Layers"}
    for i, val in enumerate(xbest, start=1):
        layers_rec[f"gamma{i}"] = val
    records.append(layers_rec)

    # tissues
    tissues = ['EW', 'TW', 'LW']
    tissue_dfs = []
    for tissue in tissues:
        csv_file = os.path.join(bestpath, tissue, f"{tissue}_gamma.csv")
        df_t = pd.read_csv(csv_file)
        df_t.columns = gamma_names_greek[:df_t.shape[1]]
        df_t.insert(0, 'Tissue', tissue)
        tissue_dfs.append(df_t)

        rec = {"Material": tissue}
        for i, val in enumerate(df_t.drop(columns='Tissue').iloc[0], start=1):
            rec[f"gamma{i}"] = val
        records.append(rec)

    df_tissues = pd.concat(tissue_dfs, ignore_index=True)
    print("\nInput values for tissues:")
    display(df_tissues.style.format("{:.3f}", subset=gamma_names_greek[:df_tissues.shape[1]-1]).hide(axis="index"))

    # Growth ring
    row0 = df_mat.iloc[0]
    gr_vals = row0[gamma_names_num].astype(float).values
    df_gr = pd.DataFrame([gr_vals], columns=gamma_names_greek)
    print("Output values of growth ring:")
    display(df_gr.style.format("{:.3f}").hide(axis="index"))
    gr_rec = {"Material": "GR"}
    for i, val in enumerate(gr_vals, start=1):
        gr_rec[f"gamma{i}"] = val
    records.append(gr_rec)

    # Save summary
    df_out = pd.DataFrame(records, columns=["Material"] + gamma_names_num)
    df_out.to_csv(outpath, index=False)
    print(f"\nBest values saved to: {outpath}")



### Plot objective funtion Vs attempts

In [ ]:
######################## Plot Objective function Vs attempts ########################
matfile = 'Results.mat'
data = loadmat(matfile)['Data']
#xbest = np.array(data['xbest'], dtype=float).flatten()
obj_fun = np.array(data['Y'], dtype=float)
x = np.arange(1, obj_fun.size + 1, dtype=int)

# Parameters
split_idx = 13
break_y  = 2
max_y = obj_fun.max() * 1.2

# Prepare plot
fig, (ax_high, ax_low) = plt.subplots(2, 1, sharex=True, figsize=(9, 6), gridspec_kw={'height_ratios': [1, 2]})
labelsize = 20

# Plot values
for ax in (ax_high, ax_low):
    if ax is ax_high:
        cond_init = obj_fun[:split_idx] > break_y
        cond_new = obj_fun[split_idx:] > break_y
    else:
        cond_init = obj_fun[:split_idx] <= break_y
        cond_new = obj_fun[split_idx:] <= break_y
    ax.scatter(x[:split_idx][cond_init], obj_fun[:split_idx][cond_init], marker='x', color='k', label='Initial designs', clip_on=False, zorder=3, s=50)
    ax.scatter(x[split_idx:][cond_new], obj_fun[split_idx:][cond_new], marker='o', color='b', label='New designs', clip_on=False, zorder=3, s=50)
    ax.grid(which='major', axis='both', alpha=0.5)

# Mark the global minimum
min_idx = obj_fun.argmin()
min_x = x[min_idx]
min_y = obj_fun[min_idx]
offset = break_y * 0.1
ax_low.scatter(min_x, min_y, marker='o', facecolors='none', edgecolors='r', s=300, zorder=4)
arrow_offset=offset*2
ax_low.annotate('', xy=(min_x,min_y), xytext=(min_x+arrow_offset,min_y+arrow_offset), arrowprops=dict(color='r',arrowstyle='->',shrinkB=10), annotation_clip=False)

# Set broken-axis limits
locator = MaxNLocator(integer=True)
ticks = locator.tick_values(break_y, obj_fun.max()*1.2)
ax_high.set_ylim(break_y, ticks[-1])
ax_low.set_ylim(0, break_y)
ax_low.set_xlim(left=1, right=50)

# Hide touching spines
ax_high.spines['bottom'].set_visible(False)
ax_low.spines['top'].set_visible(False)

# Set ticks
ax_low.xaxis.set_major_locator(MaxNLocator(integer=True))
ax_low.tick_params(labelsize=labelsize)
ax_high.tick_params(labelbottom=False, bottom=False, labelsize=labelsize)

# Set horizontal “break” marks
d = 0.015  # size in axes coords
# Upper plot
ax_high.plot((-d, +d), (-d, -d), transform=ax_high.transAxes, color='k', clip_on=False)
ax_high.plot((1-d, 1+d), (-d, -d), transform=ax_high.transAxes, color='k', clip_on=False)
# Lower plot
ax_low.plot((-d, +d), (1, 1), transform=ax_low.transAxes, color='k', clip_on=False)
ax_low.plot((1-d, 1+d), (1, 1), transform=ax_low.transAxes, color='k', clip_on=False)

# Set labels and legend
ax_low.set_xlabel('# Attempt', fontsize=labelsize)
fig.text(0., 0.5, r'$f_{obj}$', va='center', ha='center', rotation='vertical', fontsize=labelsize+2)
ax_high.legend(loc='upper left', fontsize=labelsize)

# Show plot
plt.tight_layout()
plt.show()

# Save plot
savepath = cwd + "/obj_fun_Vs_attempts.png"
fig.savefig(savepath, dpi=300, bbox_inches='tight')

### Plot master curves Vs MatSuMoTo best attempt

In [ ]:
# Set retardation times and time array
tau_meso = np.array([0.1, 1, 10, 100])  # hours
tau_macro = np.array([1, 10, 100, 1000])  # hours
times = np.arange(0, 150, 1)  # 30 days in hours

# Prepare figure
fig, ax = plt.subplots(figsize=(12, 7.5)) #
labelsize = 26
ax.grid(alpha=0.8, color="0.3")

# Grab the Paired colormap and sample as many distinct colors as you have master curves
cmap = plt.get_cmap('tab10')
n_curves = len(df_master)
colors = [cmap(i) for i in np.linspace(0, 1, n_curves)]

######################## Plot master curves ########################
for (row_idx, row), color in zip(df_master.iterrows(), colors):
    # Extract gamma coefficients
    gammas = row[['gamma1', 'gamma2', 'gamma3', 'gamma4']].astype(float).values
    # Compute normalized creep compliance
    J = np.zeros_like(times, dtype=float)
    for g, t_i in zip(gammas, tau_macro):
        J += g * (1 - np.exp(-times / t_i))
    # Label
    idx = row['D'].lstrip('D')
    label = rf'$J_{{c,{idx}}}/C_{{0,{idx}}}^{{-1}}$'
    # Plot with Paired color
    ax.plot(times, J, label=label, color=color, linewidth=3, linestyle='--')

######################## Plot MatSuMoTo curve (keep black) ########################
# (choose your row as before…)
if attempt_dirs:
    row = df_mat.iloc[0]
    gammas = row[['gamma1','gamma2','gamma3','gamma4']].astype(float).values
else:
    greek_cols = [c for c in df_out.columns if c.startswith('γ')]
    gammas = df_out.loc['GR', greek_cols].astype(float).values

J = np.zeros_like(times, dtype=float)
for g, t_i in zip(gammas, tau_meso):
    J += (1/g) * (1 - np.exp(-times / t_i))

#ax.plot(times, J, color='k', linewidth=3, linestyle='-')  # MatSuMoTo in black

# Finalize axes
ax.set_xlim(0, times.max())
ax.set_ylim(bottom=0)
ax.set_xlabel('Time [h]', fontsize=labelsize)
ax.set_ylabel(r'$J_{c,AB}/C_{0,AB}^{-1}$ [-]', fontsize=labelsize)
ax.set_ylabel(r'$J_{c}/C_{0}^{-1}$ [-]', fontsize=labelsize)
ax.tick_params(axis='both', labelsize=labelsize)

# Build legend (including a custom handle for MatSuMoTo fit)
handles, labels = ax.get_legend_handles_labels()
"""fit_label = rf'$J_{{c,11}}/C_{{0,11}}^{{-1}}$ fit'
fit_line = Line2D([], [], color='k', linestyle='-', linewidth=3, label=fit_label)
handles.append(fit_line)
labels.append(fit_label)"""

leg = ax.legend(handles, labels,
                #title='Macro-exp. data**',
                #title_fontsize=labelsize,
                fontsize=labelsize-2,
                loc='center left',
                bbox_to_anchor=(1.0, 0.5))
leg.get_title().set_ha('left')

plt.tight_layout()
plt.show()

# Save plot
savepath = cwd + "/GR-RVE_nlgeomOFF_norm_creep_compl_0.png"
fig.savefig(savepath, dpi=300, bbox_inches='tight')

In [ ]:
# Set retardation times and time array
tau_meso = np.array([0.1, 1, 10, 100])  # hours
tau_macro = np.array([1, 10, 100, 1000])  # hours
times = np.arange(0, 150, 1)  # 30 days in hours

# Prepare figure
fig, ax = plt.subplots(figsize=(7, 6))
labelsize = 18
ax.grid(alpha=0.5)

# Grab the Paired colormap and select its first color
cmap = plt.get_cmap('tab10')
master_color = cmap(0.0)

######################## Plot master curve ########################
row = df_master.iloc[0]
gammas = row[['gamma1', 'gamma2', 'gamma3', 'gamma4']].astype(float).values

# Compute normalized creep compliance
J = np.zeros_like(times, dtype=float)
for g, t_i in zip(gammas, tau_macro):
    J += g * (1 - np.exp(-times / t_i))

# Create and plot label
idx = row['D'].lstrip('D')
label = rf'$J_{{c,{idx}}}/C_{{0,{idx}}}^{{-1}}$ Macro-exp. data**'
ax.plot(times, J, label=label,
        color=master_color, linewidth=2, linestyle='--')

######################## Plot MatSuMoTo curve ########################
if attempt_dirs:
    row = df_mat.iloc[0]
    gammas = row[['gamma1','gamma2','gamma3','gamma4']].astype(float).values
else:
    greek_cols = [c for c in df_out.columns if c.startswith('γ')]
    gammas = df_out.loc['GR', greek_cols].astype(float).values

J = np.zeros_like(times, dtype=float)
for g, t_i in zip(gammas, tau_meso):
    J += (1/g) * (1 - np.exp(-times / t_i))

mat_label = rf'$J_{{c,{idx}}}/C_{{0,{idx}}}^{{-1}}$ MatS. fit'
ax.plot(times, J, color='k', linewidth=2, linestyle='-', label=mat_label)

# Finalize plot
ax.set_xlim(0, times.max())
ax.set_ylim(bottom=0)
ax.set_xlabel('Time [h]', fontsize=labelsize)
ax.set_ylabel(r'$J_{c,AB}/C_{0,AB}^{-1}$ [-]', fontsize=labelsize)
ax.tick_params(axis='both', labelsize=labelsize)
ax.legend(fontsize=labelsize, loc='lower right')

plt.tight_layout()
plt.show()

# Save plot
savepath = cwd + "/GR-RVE_nlgeomOFF_norm_creep_compl_single_C11.png"
fig.savefig(savepath, dpi=300, bbox_inches='tight')

### Plot master curves Vs all attempts